# Predicting Sale Price from NYC Property Sale Data
## Berkeley SAAS CX Fall 2020 Kaggle Competition
## Ming Fong and Yifan Zhang


Helpful:

https://www.kaggle.com/sahilrider/learn-regression-nyc#4.-Modelling

In [116]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [117]:
# Display all columns of Dataframes
pd.set_option('display.max_columns', None)

In [118]:
X_train = pd.read_csv("data/train_features.csv")
y_train = pd.read_csv("data/train_targets.csv")
X_test = pd.read_csv("data/test_features.csv")
# sample_submission = pd.read_csv("output/sample_submission.csv")

In [119]:
# X_train.set_index("id")
# y_train.set_index("id")
# X_test.set_index("id")
# X_train.head()

In [120]:
# merge sale price onto feature data
df = X_train.merge(right = y_train, on = "id", how = "outer")

# train_merged = X_train.merge(right = y_train, on = "id")

In [121]:
# drop training rows with 0 gross or land sqft
df = df.drop(df[df["GROSS SQUARE FEET"] == 0].index)
df = df.drop(df[df["LAND SQUARE FEET"] == 0].index)

In [122]:
df = pd.concat([df, X_test], axis=0, ignore_index = True)

In [123]:
df = df.set_index("id")

In [124]:
# "EASE-MENT" is an empty column, delete it
del df["EASE-MENT"]

# Delete date column
del df["SALE DATE"]

del df["LOT"]
del df["ADDRESS"]
del df["APARTMENT NUMBER"]

In [125]:
# Correlation between features

# corr = train_merged.corr()
# print(corr['SALE PRICE'].sort_values(ascending=False))
# sns.heatmap(corr)

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31209 entries, train0 to test9288
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         31209 non-null  int64  
 1   NEIGHBORHOOD                    31209 non-null  object 
 2   BUILDING CLASS CATEGORY         31209 non-null  object 
 3   TAX CLASS AT PRESENT            31209 non-null  object 
 4   BLOCK                           31209 non-null  int64  
 5   BUILDING CLASS AT PRESENT       31209 non-null  object 
 6   ZIP CODE                        31209 non-null  int64  
 7   RESIDENTIAL UNITS               31209 non-null  int64  
 8   COMMERCIAL UNITS                31209 non-null  int64  
 9   TOTAL UNITS                     31209 non-null  int64  
 10  LAND SQUARE FEET                31209 non-null  float64
 11  GROSS SQUARE FEET               31209 non-null  float64
 12  YEAR BUILT                   

In [127]:
categorical_features = ["BOROUGH", "ZIP CODE", "BUILDING CLASS CATEGORY", "TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE"]

del df["BUILDING CLASS AT PRESENT"]
del df["TAX CLASS AT PRESENT"]
# del df["BUILDING CLASS AT TIME OF SALE"]

# Too many features for location
del df["BLOCK"]
del df["NEIGHBORHOOD"]
# del df["ZIP CODE"]

In [128]:
# One-hot encoding
df = pd.get_dummies(df, columns = categorical_features)
df.info(verbose = True, memory_usage = True, null_counts = True)

             31209 non-null  uint8  
 180 ZIP CODE_11428                                                        31209 non-null  uint8  
 181 ZIP CODE_11429                                                        31209 non-null  uint8  
 182 ZIP CODE_11432                                                        31209 non-null  uint8  
 183 ZIP CODE_11433                                                        31209 non-null  uint8  
 184 ZIP CODE_11434                                                        31209 non-null  uint8  
 185 ZIP CODE_11435                                                        31209 non-null  uint8  
 186 ZIP CODE_11436                                                        31209 non-null  uint8  
 187 ZIP CODE_11691                                                        31209 non-null  uint8  
 188 ZIP CODE_11692                                                        31209 non-null  uint8  
 189 ZIP CODE_11693                                                     

# Regression

In [129]:
test_row_count = X_test.shape[0]
test_row_count

9289

In [130]:
df[:(df.shape[0] - test_row_count)].drop("SALE PRICE", 1)

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,MEDIAN INCOME,MEAN INCOME,POPULATION,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,ZIP CODE_10001,ZIP CODE_10002,ZIP CODE_10003,ZIP CODE_10004,ZIP CODE_10005,ZIP CODE_10006,ZIP CODE_10007,ZIP CODE_10009,ZIP CODE_10010,ZIP CODE_10011,ZIP CODE_10012,ZIP CODE_10013,ZIP CODE_10014,ZIP CODE_10016,ZIP CODE_10017,ZIP CODE_10018,ZIP CODE_10019,ZIP CODE_10021,ZIP CODE_10022,ZIP CODE_10023,ZIP CODE_10024,ZIP CODE_10025,ZIP CODE_10026,ZIP CODE_10027,ZIP CODE_10028,ZIP CODE_10029,ZIP CODE_10030,ZIP CODE_10031,ZIP CODE_10032,ZIP CODE_10033,ZIP CODE_10034,ZIP CODE_10035,ZIP CODE_10036,ZIP CODE_10037,ZIP CODE_10038,ZIP CODE_10039,ZIP CODE_10040,ZIP CODE_10044,ZIP CODE_10065,ZIP CODE_10075,ZIP CODE_10128,ZIP CODE_10301,ZIP CODE_10302,ZIP CODE_10303,ZIP CODE_10304,ZIP CODE_10305,ZIP CODE_10306,ZIP CODE_10307,ZIP CODE_10308,ZIP CODE_10309,ZIP CODE_10310,ZIP CODE_10312,ZIP CODE_10314,ZIP CODE_10451,ZIP CODE_10452,ZIP CODE_10453,ZIP CODE_10454,ZIP CODE_10455,ZIP CODE_10456,ZIP CODE_10457,ZIP CODE_10458,ZIP CODE_10459,ZIP CODE_10460,ZIP CODE_10461,ZIP CODE_10462,ZIP CODE_10463,ZIP CODE_10464,ZIP CODE_10465,ZIP CODE_10466,ZIP CODE_10467,ZIP CODE_10468,ZIP CODE_10469,ZIP CODE_10470,ZIP CODE_10471,ZIP CODE_10472,ZIP CODE_10473,ZIP CODE_10474,ZIP CODE_10475,ZIP CODE_10803,ZIP CODE_11001,ZIP CODE_11004,ZIP CODE_11005,ZIP CODE_11040,ZIP CODE_11101,ZIP CODE_11102,ZIP CODE_11103,ZIP CODE_11104,ZIP CODE_11105,ZIP CODE_11106,ZIP CODE_11201,ZIP CODE_11203,ZIP CODE_11204,ZIP CODE_11205,ZIP CODE_11206,ZIP CODE_11207,ZIP CODE_11208,ZIP CODE_11209,ZIP CODE_11210,ZIP CODE_11211,ZIP CODE_11212,ZIP CODE_11213,ZIP CODE_11214,ZIP CODE_11215,ZIP CODE_11216,ZIP CODE_11217,ZIP CODE_11218,ZIP CODE_11219,ZIP CODE_11220,ZIP CODE_11221,ZIP CODE_11222,ZIP CODE_11223,ZIP CODE_11224,ZIP CODE_11225,ZIP CODE_11226,ZIP CODE_11228,ZIP CODE_11229,ZIP CODE_11230,ZIP CODE_11231,ZIP CODE_11232,ZIP CODE_11233,ZIP CODE_11234,ZIP CODE_11235,ZIP CODE_11236,ZIP CODE_11237,ZIP CODE_11238,ZIP CODE_11239,ZIP CODE_11354,ZIP CODE_11355,ZIP CODE_11356,ZIP CODE_11357,ZIP CODE_11358,ZIP CODE_11360,ZIP CODE_11361,ZIP CODE_11362,ZIP CODE_11363,ZIP CODE_11364,ZIP CODE_11365,ZIP CODE_11366,ZIP CODE_11367,ZIP CODE_11368,ZIP CODE_11369,ZIP CODE_11370,ZIP CODE_11372,ZIP CODE_11373,ZIP CODE_11374,ZIP CODE_11375,ZIP CODE_11377,ZIP CODE_11378,ZIP CODE_11379,ZIP CODE_11385,ZIP CODE_11411,ZIP CODE_11412,ZIP CODE_11413,ZIP CODE_11414,ZIP CODE_11415,ZIP CODE_11416,ZIP CODE_11417,ZIP CODE_11418,ZIP CODE_11419,ZIP CODE_11420,ZIP CODE_11421,ZIP CODE_11422,ZIP CODE_11423,ZIP CODE_11426,ZIP CODE_11427,ZIP CODE_11428,ZIP CODE_11429,ZIP CODE_11432,ZIP CODE_11433,ZIP CODE_11434,ZIP CODE_11435,ZIP CODE_11436,ZIP CODE_11691,ZIP CODE_11692,ZIP CODE_11693,ZIP CODE_11694,BUILDING CLASS CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING CLASS CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING CLASS CATEGORY_03 THREE FAMILY DWELLINGS,BUILDING CLASS CATEGORY_04 TAX CLASS 1 CONDOS,BUILDING CLASS CATEGORY_05 TAX CLASS 1 VACANT LAND,BUILDING CLASS CATEGORY_06 TAX CLASS 1 - OTHER,BUILDING CLASS CATEGORY_07 RENTALS - WALKUP APARTMENTS,BUILDING CLASS CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING CLASS CATEGORY_09 COOPS - WALKUP APARTMENTS,BUILDING CLASS CATEGORY_10 COOPS - ELEVATOR APARTMENTS,BUILDING CLASS CATEGORY_11 SPECIAL CONDO BILLING LOTS,BUILDING CLASS CATEGORY_11A CONDO-RENTALS,BUILDING CLASS CATEGORY_12 CONDOS - WALKUP APARTMENTS,BUILDING CLASS CATEGORY_13 CONDOS - ELEVATOR APARTMENTS,BUILDING CLASS CATEGORY_14 RENTALS - 4-10 UNIT,BUILDING CLASS CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL,BUILDING CLASS CATEGORY_16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT,BUILDING CLASS CATEGORY_17 CONDO COOPS,BUILDING CLASS CATEGORY_21 OFFICE BUILDINGS,BUILDING CLASS CATEGORY_22 STORE BUILDINGS,BUILDING CLASS CATEGORY_23 LOFT BUILDINGS,BUILDING CLASS CATEGORY_25 LUXURY HOTELS,BUILDING CLASS CATEGORY_26 OTHER HOTELS,BUILDING CLASS CATEGORY_27 FACTORIES,BUILD

In [131]:
# Split train and test data
from sklearn.model_selection import train_test_split

y = df["SALE PRICE"][:(df.shape[0] - test_row_count)]
X = df[:(df.shape[0] - test_row_count)].drop("SALE PRICE", 1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [132]:
from sklearn.metrics import mean_squared_error
# Root mean squared error
def rmse(y_test,y_pred):
      return np.sqrt(mean_squared_error(y_test,y_pred))

In [133]:
# linear regression
linreg = linear_model.LinearRegression()
linreg.fit(X_train, y_train)
y_pred_lin = linreg.predict(X_valid)
rmse(y_valid, y_pred_lin)

16597434.487994341

In [134]:
linear_submission = pd.DataFrame(linreg.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
linear_submission.index = X_test["id"]
linear_submission.columns = ["SALE PRICE"]
linear_submission.to_csv("output/linear_submission.csv")
linear_submission.head()

,SALE PRICE
id,
test0,1.381722e+07
test1,-7.227431e+06
test2,3.219743e+05
test3,-6.935860e+06
test4,-8.531378e+05


In [135]:
# Lasso regression
alpha = 12
lasso_regr = linear_model.Lasso(alpha = alpha, max_iter = 500)
lasso_regr.fit(X_train, y_train)
Y_pred_lasso = lasso_regr.predict(X_valid)
rmse(y_valid,Y_pred_lasso)

16590548.748000665

In [136]:
lasso_submission = pd.DataFrame(lasso_regr.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
lasso_submission.index = X_test["id"]
lasso_submission.columns = ["SALE PRICE"]
# lasso_submission[lasso_submission["SALE PRICE"] < 1000] = y_train.median()       # hard code set < 1000 to median
lasso_submission.to_csv("output/lasso_submission.csv")
lasso_submission.head()

,SALE PRICE
id,
test0,1.383307e+07
test1,1.410426e+06
test2,3.266453e+05
test3,1.697883e+06
test4,7.809926e+06


In [137]:
# Ridge regression
ridge = linear_model.Ridge(alpha = 0.01, normalize = True)
ridge.fit(X_train, y_train)
Y_pred_ridge = ridge.predict(X_valid)
rmse(y_valid, Y_pred_ridge)

16527272.936116837

In [138]:
ridge_submission = pd.DataFrame(ridge.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
ridge_submission.index = X_test["id"]
ridge_submission.columns = ["SALE PRICE"]
ridge_submission.to_csv("output/ridge_submission.csv")
ridge_submission.head()

,SALE PRICE
id,
test0,1.363452e+07
test1,1.129067e+06
test2,3.349095e+05
test3,1.274455e+06
test4,7.513601e+06


In [139]:
from sklearn.ensemble import RandomForestRegressor

In [140]:
# Random forest
rf_regr = RandomForestRegressor()
rf_regr.fit(X_train, y_train)
Y_pred_rf = rf_regr.predict(X_valid)
rmse(y_valid, Y_pred_rf)

16398086.072808791

In [141]:
rf_submission = pd.DataFrame(rf_regr.predict(df[(df.shape[0] - test_row_count):].drop("SALE PRICE", 1)))
rf_submission.index = X_test["id"]
rf_submission.columns = ["SALE PRICE"]
rf_submission.to_csv("output/rf_submission.csv")
rf_submission.head()

,SALE PRICE
id,
test0,9.233541e+06
test1,8.255907e+05
test2,5.404746e+05
test3,1.703580e+06
test4,2.653210e+06


## Bad Linear Regression

In [85]:
# X = X_train[["LAND SQUARE FEET", "GROSS SQUARE FEET", "MEDIAN INCOME", "MEAN INCOME"]]
# Y = y_train["SALE PRICE"]

In [86]:
# model = linear_model.LinearRegression()
# model.fit(X,Y)
# print('Intercept: \n', model.intercept_)
# print('Coefficients: \n', model.coef_)

In [87]:
# x_test_predictions = pd.DataFrame(model.predict(X_test[["LAND SQUARE FEET", "GROSS SQUARE FEET", "MEDIAN INCOME", "MEAN INCOME"]]))
# x_test_predictions.index = X_test["id"]
# x_test_predictions.columns = ["SALE PRICE"]
# x_test_predictions.to_csv("output/submission.csv")
# x_test_predictions.head()

# Submit to Kaggle
Run this command to send a submission to the Kaggle competition

In [112]:
# !kaggle competitions submit -c saas-2020-fall-cx-kaggle-compeition -f output/lasso_submission.csv -m "lasso remove zero gross sqft and land sqft"

Successfully submitted to SAAS 2020 Fall CX Kaggle Compeition

  0%|          | 0.00/257k [00:00<?, ?B/s]
  3%|▎         | 8.00k/257k [00:00<00:03, 78.8kB/s]
 34%|███▍      | 88.0k/257k [00:00<00:01, 107kB/s] 
 44%|████▎     | 112k/257k [00:00<00:01, 102kB/s] 
 53%|█████▎    | 136k/257k [00:00<00:01, 99.4kB/s]
 59%|█████▉    | 152k/257k [00:00<00:01, 97.9kB/s]
 65%|██████▌   | 168k/257k [00:01<00:00, 94.7kB/s]
 72%|███████▏  | 184k/257k [00:01<00:00, 92.9kB/s]
 78%|███████▊  | 200k/257k [00:01<00:00, 94.5kB/s]
 84%|████████▍ | 216k/257k [00:01<00:00, 91.3kB/s]
 90%|█████████ | 232k/257k [00:01<00:00, 93.3kB/s]
 97%|█████████▋| 248k/257k [00:02<00:00, 91.6kB/s]
100%|██████████| 257k/257k [00:05<00:00, 52.4kB/s]
